# 01: Opening and understanding

In [1]:
library(IRdisplay)
display_html('<iframe width="560" height="315" src="https://www.youtube.com/embed/Xer1XBm3sns" frameborder="0" allowfullscreen></iframe>')

In this notebook we will go through how to open a NetCDF file in R. We will also try to understand the file and extract the data and metadata from it.

Firstly, let’s import the modules that we will work with.

In [2]:
if (!requireNamespace("RNetCDF", quietly = TRUE)) {
  install.packages("RNetCDF")
}
library(RNetCDF)

You can find the documentation for the RNetCDF package here:
https://cran.r-project.org/web/packages/RNetCDF/RNetCDF.pdf

The syntax for functions in RNetCDF is very repetitive as you will learn throughout this course, which makes it easy to learn. However, there are other libraries you can use like ncdf4.
https://cran.r-project.org/web/packages/ncdf4/ncdf4.pdf

In this tutorial, we will stick to RNetCDF.

## Importing some data from OPeNDAP

Have you heard of OPeNDAP? Try running the cell below.

OPeNDAP, which stands for “Open-source Project for a Network Data Access Protocol,” makes it easier to access and share scientific data over the internet. One advantage of using OPeNDAP is that you don’t need to download the data to use them!

In [3]:
netcdf_file <- 'https://www.ncei.noaa.gov/thredds/dodsC/noaa-global-temp-v5/NOAAGlobalTemp_v5.0.0_gridded_s188001_e202212_c20230108T133308.nc'
data <- open.nc(netcdf_file)

If we want some quick information about the file we can make an inquiry

In [4]:
file.inq.nc(data)

$ndims
[1] 4

$nvars
[1] 5

$ngatts
[1] 66

$unlimdimid
[1] NA

$format
[1] "classic"

$libvers
[1] "4.8.1 of Sep 29 2021 09:36:14 $"

The inquiry is a list of key value pairs, so we can access the values like this

In [5]:
file.inq.nc(data)['format']

$format
[1] "classic"

Though often we will want more information, so we can do this instead:

In [6]:
print.nc(data)

netcdf classic {
dimensions:
	lat = 36 ;
	lon = 72 ;
	time = 1716 ;
	z = 1 ;
variables:
	NC_FLOAT time(time) ;
		NC_CHAR time:long_name = "reference time of global temperature anomalies" ;
		NC_CHAR time:standard_name = "time" ;
		NC_CHAR time:coverage_content_type = "coordinate" ;
		NC_CHAR time:units = "days since 1800-01-01 00:00:00" ;
		NC_CHAR time:calendar = "gregorian" ;
		NC_CHAR time:axis = "T" ;
	NC_FLOAT lat(lat) ;
		NC_CHAR lat:long_name = "Latitude" ;
		NC_CHAR lat:standard_name = "latitude" ;
		NC_CHAR lat:coverage_content_type = "coordinate" ;
		NC_CHAR lat:units = "degrees_north" ;
		NC_CHAR lat:grids = "Uniform grid from -87.5 to 87.5 by 5" ;
		NC_FLOAT lat:valid_min = -87.5 ;
		NC_FLOAT lat:valid_max = 87.5 ;
		NC_CHAR lat:axis = "Y" ;
		NC_CHAR lat:_CoordinateAxisType = "Lat" ;
		NC_CHAR lat:coordinate_defines = "center" ;
	NC_FLOAT lon(lon) ;
		NC_CHAR lon:long_name = "Longitude" ;
		NC_CHAR lon:standard_name = "longitude" ;
		NC_CHAR lon:coverage_content_type = "co

Let’s break down what we see above.

A classic NetCDF file like this one can be broken down into 3 components - dimensions, variables and attributes.

* Dimensions: Define the size of the variables.
* Variables: Where the data are stored. The variables can be broken down into coordinate variables and data variables. In RNetCDF, the coordinate variables and data variables are displayed together, but if you open a NetCDF file using different software they might be separated.
    * Coordinate variables: Coordinate variables are things like longitude, latitude, time and elevation/depth. The first variable listed is *time* which has 1 dimension with the same name in brackets. So we know that the *time* variable is a 1D array with 1716 data points. It is common practice for coordinate variables to have the same name as their respective dimension, but this might not always be the case.
    * Data variables: This file has only one data variable, *anom*. This has 4 dimensions, so we can see that the data are stored in a 4D array with size 72 x 36 x 1 x 1716. 
* Attributes: The metadata describing the data. A CF-NetCDF file should have both global and variable attributes.
    * Variable attributes: Describe each variable
    * Global attributes: Describe the whole dataset (e.g. title, creator, keywords, a bounding box for the coordinates)

We will now have a closer look at how to access each of these components, starting at the bottom with attributes.

## Attributes

Accessing an attribute using RNetCDF can be done like

`att.get.nc(nc_object, variable_name, attribute_name)`

### Global attributes

For a global attribute, we assign a special variable name, *NC_GLOBAL*.

`att.get.nc(nc_object, "NC_GLOBAL", attribute_name)`

If we want to just access the *creator_name* attribute for example:

In [7]:
att.get.nc(data, "NC_GLOBAL", "creator_name")

[1] "Boyin Huang, Xungang Yin, Ph.D"

If you want more information than just the value, you can make an inquiry. See how the syntax is almost the same for 'getting' and attribute as it is for 'inquiring' about one? 

In [8]:
att.inq.nc(data, "NC_GLOBAL", "creator_name")

$id
[1] 16

$name
[1] "creator_name"

$type
[1] "NC_CHAR"

$length
[1] 30

*Conventions* is probably the most important global attribute because it tells you (and a machine) how to interpret the rest of the file.

In [9]:
att.get.nc(data, "NC_GLOBAL", "Conventions")

[1] "CF-1.6, ACDD-1.3"

*CF-1.6* refers to version 1.6 of the CF conventions, which you can find here:

https://cfconventions.org/ https://cfconventions.org/Data/cf-conventions/cf-conventions-1.6/cf-conventions.html

The CF conventions are a set of standards that define how a NetCDF file should be structured. The document linked above is extensive, but the aim is to provide a standardised way to organise many different types of data. You don’t need to read it all, but it should be your go-to place if you want to know how to structure or understand a CF-NetCDF file, or encode certain types of data.

However, the CF conventions are light on discovery metadata. Discovery metadata are metadata that can be used to find data. For example, when and where the data were collected and by whom, some keywords etc. So we also use the *ACDD* convention - The Attribute Convention for Data Discovery.

https://wiki.esipfed.org/Attribute_Convention_for_Data_Discovery_1-3

In most cases, if you want to find out what a global attribute means, you can visit the ACDD convention page above to find a description of the attribute. There are other conventions that someone might have included that you can also find online, but we recommend that you always follow the CF and ACDD conventions as a minimum when creating a NetCDF file.

The person who created this file should have read the relevant sections of these documents to make sure that the files comply with these conventions. There are also validators you can run your files by to make sure that you file is compliant with the conventions before you publish it. For example:

https://compliance.ioos.us/index.html

By following conventions, the data creator and user, human or machine, should be able to understand the data in the same way. A NetCDF file itself is not neccessarily FAIR because you could include any attributes or structure your data however you like. A CF-NetCDF file is FAIR.

### Variable attributes

Let's extract a variable attribute now from the *anom* variable:

In [10]:
att.get.nc(data, "anom", "standard_name")

[1] "surface_temperature_anomaly"

The variable name *anom* is not standardised. Fortunately, the *standard_name* variable attribute is standardised. You can find sea_water_temperature in the list of CF standard names here. 
https://cfconventions.org/Data/cf-standard-names/current/build/cf-standard-name-table.html

Hopefully, the data creator has selected the *standard_name* from the CF standard name table after reading the description for that term. You, the data user, can now also find that term and its description. This way, the data provider and the data user can share the same understanding about what the these data are. Any other CF-NetCDF file that contains the same type of data should also include the same *standard_name*. Controlled vocabularies like this one are understandable to machines too! 

The data in the file doesn’t need to be stored with the same units, but should be stored with units that are physically equivalent. For example, in this case the units degrees_C are physically equivalent to K.

## Dimensions

To extract information about a dimension, you can do this:

In [11]:
dim.inq.nc(data,'lat')

$id
[1] 0

$name
[1] "lat"

$length
[1] 36

$unlim
[1] FALSE

If you didn't know the name of the dimension, you could use the index of the variable to call it instead. Remember that 0 is always the first.

In [12]:
dim.inq.nc(data,0)

$id
[1] 0

$name
[1] "lat"

$length
[1] 36

$unlim
[1] FALSE

The *lat* dimension has 36 points. Dimensions tell you about the shape and size of your variables. In this case, we know that any variable with a dimension of only *lat* will have 36 data points - though some could be NaN. The dimension doesn't tell you anything about what the values are - that information is in the variables.

## Variables

Finally, the most important part - getting at the data!

If you know the variable name, you can get the data like this:

In [13]:
var.get.nc(data,"lat")

[1] -87.5 -82.5 -77.5 -72.5 -67.5 -62.5 -57.5 -52.5 -47.5 -42.5 -37.5 -32.5
[13] -27.5 -22.5 -17.5 -12.5  -7.5  -2.5   2.5   7.5  12.5  17.5  22.5  27.5
[25]  32.5  37.5  42.5  47.5  52.5  57.5  62.5  67.5  72.5  77.5  82.5  87.5

Or inquire about it

In [14]:
var.inq.nc(data, "lat")

$id
[1] 1

$name
[1] "lat"

$type
[1] "NC_FLOAT"

$ndims
[1] 1

$dimids
[1] 0

$natts
[1] 10

We can get just a subset of the data. For example, to get 3 values starting from the 5th value in the array, where *start* is the first value to extract and *count* is how many values:

In [15]:
var.get.nc(data, "lat", start=c(5), count=c(3))

[1] -67.5 -62.5 -57.5

There is a lot of data in the anom variable, so we won't display all of that below! But let's try and extract data for a single date. 

Let's again first select based on the index using *start* and *count. The order in the lists corresponds to the order of the dimensions that the variable has. In this case, we are extracting data for all latitudes and longitudes (so *NA* is used) and from the 1st (and only) vertical coordinate. I have selected data for an arbitrary time slice, 700 in this case.

In [16]:
lat <- var.get.nc(data, 'lat')
lon <- var.get.nc(data, 'lon')
anom <- var.get.nc(data, 'anom', start=c(NA, NA, 1, 700), count=c(NA,NA,1,1))
anom

NA,NA,NA,NA,NA,NA,-0.6202122,-0.28983244,0.127612844,0.105352372,⋯,-0.537931740,-1.87294960,0.24818414,0.49265805,0.28210214,0.449737489,1.1294684,0.9146796,NA,NA
NA,NA,NA,NA,NA,NA,-0.4854550,-0.29407427,0.026559843,0.000770571,⋯,-0.835091114,-2.66000009,-1.08594847,0.69367015,0.10752933,0.591097713,1.1051759,0.7053886,NA,NA
NA,NA,NA,NA,NA,NA,-0.7393103,-0.28969392,0.041384175,-0.090124525,⋯,-1.269892812,-3.05221272,-1.84167469,0.43207514,0.35837406,0.443900764,0.9841446,0.8509724,NA,NA
NA,NA,NA,NA,NA,NA,-0.8334374,-0.22463292,0.190988734,-0.151320428,⋯,-1.601708531,-3.17991972,-2.66849828,0.27449751,0.36497775,0.345626265,1.0454960,NA,NA,NA
NA,NA,NA,NA,NA,NA,-0.8196298,-0.20011082,0.178636834,-0.218324393,⋯,-2.145020962,-2.74029255,-2.38935113,-0.28059667,0.50999999,0.259044379,1.0828582,NA,NA,NA
NA,NA,NA,NA,NA,NA,-0.5987538,-0.09562428,0.127850562,-0.302159876,⋯,-1.020416260,-1.85013807,-2.57099295,-1.79307616,-0.99419832,0.613359451,0.9467384,NA,NA,NA
NA,NA,NA,NA,NA,NA,-0.4492744,-0.14773342,0.020136885,-0.136485785,⋯,-0.401134670,-1.10070395,-2.19099236,-1.69648826,-0.55620253,0.760314703,0.8679525,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,0.009670788,-0.005256775,⋯,-0.638019681,-1.44743681,-1.64928043,-1.89835858,-1.24357367,1.259999990,0.7236950,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,NA,-0.158918560,-0.102225631,⋯,-0.917013168,-1.16364527,-0.99239308,-1.57622480,-0.27919686,-0.005481333,-0.1899090,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,-0.64733446,-0.345341444,-0.261111975,⋯,-0.276104450,-0.50022191,0.31061247,-0.72260940,0.48036933,4.380000114,NA,NA,NA,NA
NA,NA,NA,NA,NA,NA,NA,-0.67590743,-0.373588830,-0.263846487,⋯,-0.268124402,0.10986871,1.17465675,0.57172859,0.61704886,1.343866348,2.6400001,NA,NA,NA


But in most cases, we want to select based on time (or whichever coordinate), not based on the index. So let's now select based on a given time. The units for time are

In [17]:
att.get.nc(data, "time", "units")

[1] "days since 1800-01-01 00:00:00"

There are specific recommendations on how time should be stored in CF-NetCDF files. I will try to explain briefly here, and there is a nice explanation here too: https://www.unidata.ucar.edu/software/netcdf/time/recs.html

The *time* variable has units that count from a user defined origin, for example "hours since 2020-01-01 00:00 UTC" or "days since 2014-01-01". The units may be in years, days, seconds, nanoseconds, etc. Whilst this approach may seem strange at a glance, it allows the times to be stored in conventional numerical formats such as integers or floats, and to our desired precision. This is much more efficient than using a long timestamp string for each coordinate.

Some softwares know how to interpret this and will convert the data into timestamps in when you extract the data from a CF-NetCDF file. For example, xarray in Python does this, so does Panoply. Unfortunately, RNetCDF does not - at least not at the time of writing.

So now let's extract the time series, work out what index in the time series our desired date corresponds to, and then select from the *anom* variable based on that index.

In [18]:
desired_date <- as.Date('2020-01-01')
days_since_1800 <- as.numeric(difftime(desired_date, as.Date('1800-01-01'), units = 'days'))

time <- var.get.nc(data, "time") 
# Finding index of the value
time_index <- which(time == days_since_1800)

anom <- var.get.nc(data, 'anom', start=c(NA, NA, 1, time_index), count=c(NA,NA,1,1))
anom

NA,NA,NA,-0.09,-0.4774149,-0.28590867,-0.147390142,0.162335649,0.3555031419,0.085328840,⋯,0.9572873,2.7277462,1.795067,1.0600646,0.9244725,0.901413739,0.7911479,1.0034733,NA,NA
NA,NA,NA,NA,-0.5136998,-0.28014940,-0.160545558,0.158801332,0.3941240311,0.155463889,⋯,0.7803320,2.8799996,2.655742,2.0247676,2.4968357,0.878529668,0.6843208,0.8648409,NA,NA
NA,NA,NA,0.30,-0.4267291,-0.27559584,-0.183748201,0.202584669,0.4098227322,0.211597517,⋯,1.1051872,2.7000260,3.316797,3.3139091,6.5279703,1.357819557,0.7310358,0.5478907,NA,NA
NA,NA,NA,NA,-0.2727111,-0.23345284,-0.168438852,0.203091472,0.3795720339,0.154737383,⋯,1.1962049,1.5803304,4.079368,2.4789727,5.1615229,4.826246738,0.8632154,-0.7200000,NA,NA
NA,NA,NA,NA,-0.1808161,-0.16997650,-0.113978557,0.188084275,0.3441761434,0.155692816,⋯,1.5916607,0.6660571,5.440616,2.8465347,3.7294011,4.428432941,0.8453974,NA,NA,NA
NA,NA,NA,NA,-0.1048512,-0.08315404,-0.078388907,0.161499396,0.3325377405,0.242152914,⋯,1.3523049,3.2296455,6.082641,6.5513883,7.1897578,3.872640371,0.7872372,-0.9600000,NA,NA
NA,NA,NA,NA,-0.2039971,-0.03734734,-0.050673340,0.083849691,0.2672915161,0.320076883,⋯,1.2378309,3.4596205,6.688095,8.1772079,7.9278369,3.297797918,0.7249542,NA,NA,NA
NA,NA,2.24,NA,0.4100000,0.04984033,-0.030647313,0.002807403,0.2073578984,0.060383327,⋯,1.0983480,3.6145263,7.006868,8.3200865,7.6957769,2.810969353,0.2470920,NA,NA,NA
NA,NA,NA,1.85,NA,0.09618616,-0.021199374,-0.029650999,-0.0445042588,-0.036017980,⋯,1.8151871,5.7194242,7.489800,8.2420597,7.5445147,1.702046871,0.3923281,NA,NA,NA
NA,NA,NA,NA,0.0800000,0.09327671,0.001323494,-0.059333336,-0.1571347415,-0.177169263,⋯,2.0461519,5.5822577,7.721563,8.3688650,7.5512500,2.840023041,0.4435947,NA,NA,NA
NA,NA,NA,NA,NA,0.10112605,0.023530630,-0.102159321,-0.1461249292,-0.315625727,⋯,2.3329496,5.2566156,7.872991,8.2446203,6.9870377,4.032630920,0.8986476,NA,NA,NA


If you don't know the variable name, you can use the index instead.

In [19]:
var.inq.nc(data, 2)

$id
[1] 2

$name
[1] "lon"

$type
[1] "NC_FLOAT"

$ndims
[1] 1

$dimids
[1] 1

$natts
[1] 10

## Citing data

Remember, if you use a scientific dataset in a publication you should cite it in the same way you cite a paper - in the list of references. You can also include a statement in the data availability statement, but this should be as well as citing the dataset in the list of references.

This is the recommended citation for the data used in this tutorial - remember to update the data accessed:

H.-M. Zhang, B. Huang, J. H. Lawrimore, M. J. Menne, and T. M. Smith (2019): NOAA Global Surface Temperature Dataset (NOAAGlobalTemp), Version 5.0. NOAA National Centers for Environmental Information. doi:10.25921/9qth-2p70 Accessed 2024-04-10.

## How to cite this course

If you think this course contributed to the work you are doing, consider citing it in your list of references. Here is a recommended citation:

Marsden, L. (2024, May 31). NetCDF in R - from beginner to pro. Zenodo. https://doi.org/10.5281/zenodo.11400754

And you can navigate to the publication and export the citation in different styles and formats by clicking the icon below.

You can view the Jupyter book at https://lhmarsden.github.io/NetCDF_in_R_from_beginner_to_pro

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.11400754.svg)](https://doi.org/10.5281/zenodo.11400754)